In [ ]:
!pip install google-play-scraper app-store-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: requests
    Found existing installation: requ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


\# Get The Google Play Store Comments Using Google play Scrapper


In [ ]:
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all



#For google_play_store comments
google_play_comments = reviews_all(
    'com.cnbc.client',
    sleep_milliseconds=0,
    lang='en',
    country='us',

)
#Preview of one Google Play Store Review
print("Preview")
print(google_play_comments[0])


Preview
{'reviewId': '25b15b00-ec6f-4b05-8b07-a0f8672955b5', 'userName': 'A Google user', 'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g', 'content': 'Very comprehensive on the moment info. 👍', 'score': 5, 'thumbsUpCount': 0, 'reviewCreatedVersion': '6.3.0', 'at': datetime.datetime(2024, 11, 15, 18, 36, 19), 'replyContent': None, 'repliedAt': None, 'appVersion': '6.3.0'}


# Get The App Store Comments Using  App Store Scrapper

In [ ]:
from app_store_scraper import AppStore
app = AppStore(country ='us', app_name="cnbc-stock-market-business", app_id="398018310")
# app = AppStore(country ='us', app_name="fox-business-invested-in-you", app_id="333177961")
# app = AppStore(country ='us', app_name="bloomberg-business-news-daily", app_id="281941097")

#1000 reviews
app.review(how_many= 100,sleep=1)

app_store_reviews = app.reviews

#Preview of one AppStore Review
print("Preview")
print(app_store_reviews[0])

Preview
{'date': datetime.datetime(2022, 4, 25, 13, 9, 14), 'review': 'This is a pretty good app. I use it a lot for tracking business news and reports as well as real time stock quotes and pre and post market prices. It gives live stock price  movements too. They also usually have fairly good economic news coverage as far as reporting on actual economic report releases as they occur. This is very helpful if you are looking for updates as they happen as may regard a currency trade. When I was younger my father would take me to the local brokerage house to watch the stock quotes scroll by on a thin long device mounted on the wall and talk to the broker to see and find out about real time quotes. They were not remotely available on a small device that I keep in my pocket throughout the day. What a difference.', 'rating': 5, 'isEdited': False, 'userName': 'CGudinsky', 'title': 'Decent and Reliable'}


# Transform Comments into a Pandas DataFrame

In [ ]:
import pandas as pd
google_df = pd.DataFrame(google_play_comments, columns=['at','content','score'])
appStore_df = pd.DataFrame(app_store_reviews, columns=['date','review','rating'])

google_df['at'] = pd.to_datetime(google_df['at'])

#Select reviews between 2024 and 2021
google_df = google_df[(google_df['at'].dt.year>=2021)& (google_df['at'].dt.year <= 2024)]


#Select reviews with rating 1 and 2
appStore_df = appStore_df[(appStore_df['rating'] >= 1) & (appStore_df['rating'] <= 2)]

google_df.head()


,at,content,score
0,2024-11-15 18:36:19,Very comprehensive on the moment info. 👍,5
1,2024-11-14 17:31:29,App is great but please make a dark mode for app.,5
2,2024-11-14 17:02:14,This is the best app for traders. It tells you...,5
3,2024-11-05 06:13:31,All good.,5
4,2024-11-01 17:05:32,Good,5


# Clean the Comments

In [ ]:
import re

def clean_comment(comment):
    #Remove unwanted symbols and emojis from the comments
    return re.sub(r"[^a-zA-Z0-9!?',.]", " ", comment).strip()

google_df['content'] = google_df['content'].apply(lambda x: clean_comment(str(x)) if pd.notnull(x) else "")
appStore_df['review'] = appStore_df['review'].apply(lambda x: clean_comment(str(x)) if pd.notnull(x) else "")

google_df = google_df[google_df['content']!= ""]

google_df.reset_index(drop= True, inplace=True)





# Load the tokenizer and Fine-Tuned Model

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer



tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

emotions_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=12)

#Model for the emotions
emotions_model.load_weights('/content/drive/MyDrive/EmotionCommentClassificationv2_weights.h5')

app_issues_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=12)

#Model for the app Issues
app_issues_model.load_weights('/content/drive/MyDrive/ProblemCommentClassification_weights.h5')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenize the comments


---



In [ ]:


# Function to tokenize new comments
def tokenize_new_comments(new_comments):
    return tokenizer(
        new_comments,
        padding=True,
        truncation=True,
        return_tensors='tf'
    )

token_google_comments =  tokenize_new_comments(google_df['content'].tolist())

token_app_store_comments = tokenize_new_comments(appStore_df['review'].tolist())



# Send The Tokens To The Models For Emotions Classification


In [ ]:
import numpy as np

def predict_category(logits, threshold):

  # logits = tf.convert_to_tensor(logits, dtype=tf.float32)

  max_values = tf.reduce_max(logits, axis=-1)

  max_indices = tf.argmax(logits, axis=-1)

  predicted_categories = tf.where(max_values < threshold, 12, max_indices)

  return predicted_categories

# Create a TensorFlow dataset from the tokenized data
#Dividing the reviews into batches
batch_size = 256  # Adjust as needed based on memory capacity
dataset = tf.data.Dataset.from_tensor_slices((
    token_google_comments['input_ids'],
    token_google_comments['attention_mask']
))

# Batch the dataset
dataset = dataset.batch(batch_size)

# List to store all predictions
all_emotions_predictions = []

# Iterate over the batches and make predictions
for input_ids, attention_mask in dataset:
    batch_predictions = emotions_model.predict({
        'input_ids': input_ids,
        'attention_mask': attention_mask
    })


    predicted_labels = predict_category(batch_predictions.logits, 3.5).numpy()
    all_emotions_predictions.append(predicted_labels)


flattened_predections = np.concatenate(all_emotions_predictions)



6/6 [==============================] - 4s 278ms/step


# Send The Tokens To The Model For App Issue Classification


In [ ]:
#App Issues Comments

app_issue_predictions = app_issues_model.predict(token_app_store_comments['input_ids'])


1/1 [==============================] - 3s 3s/step


# Output the Classified Comments into a Excel File

In [ ]:
import numpy as np

def predict_category(logits, threshold):

  # logits = tf.convert_to_tensor(logits, dtype=tf.float32)

  max_values = tf.reduce_max(logits, axis=-1)

  max_indices = tf.argmax(logits, axis=-1)

  predicted_categories = tf.where(max_values < threshold, 12, max_indices)

  return predicted_categories




emotions_predicted_labels = flattened_predections

app_issue_predicted_labels = predict_category(app_issue_predictions.logits, 1.0).numpy()

emotions = ['Contentment','Happy but concerned','Happy','Gratitude','Overjoyed','Euphoria','sad','Dislike','angry','Disappointment','Frustrating','Horrifying','Other']

app_issues= ['Charts','Ads','Live Tv','Audio','App Crash','Login','Performance','Watchlist','Disliked Features','Content','Expensive Subscription','Privacy','Other']


google_comments = google_df['content'].tolist()

app_issue_comments = appStore_df['review'].tolist()

emotions_predicted_categories = []

app_issues_categories = []

emotion_comment_category = []

app_issue_comment_category = []

for index , i in enumerate(emotions_predicted_labels):
  emotions_predicted_categories.append(emotions[i])
  obj = {}
  obj['comment'] = google_comments[index]
  obj['emotion']= emotions[i]
  emotion_comment_category.append(obj)

for index, i in enumerate(app_issue_predicted_labels):
  app_issues_categories.append(app_issues[i])
  obj = {}
  obj['comment'] = app_issue_comments[index]
  obj['issue'] = app_issues[i]
  app_issue_comment_category.append(obj)


emtotionsComments_df = pd.DataFrame(emotion_comment_category)
app_issues_df = pd.DataFrame(app_issue_comment_category)



emtotionsComments_df.to_excel('ClassifiedEmotionsComments.xlsx', index=False)
app_issues_df.to_excel('ClassifiedAppProblems.xlsx', index=False)

print("Comments are Classified.")




Comments are Classified.
